In [ ]:
#| hide
!pip install -Uqq nixtla

In [ ]:
#| hide 
from nixtla.utils import in_colab

In [ ]:
#| hide 
IN_COLAB = in_colab()

In [ ]:
#| hide
if not IN_COLAB:
    from nixtla.utils import colab_badge
    from dotenv import load_dotenv

# Add exogenous variables

To model with exogenous features, you have two options:
1. Use historical exogenous variables: include these variables in the DataFrame you pass to the `forecast` method
2. Use future exogenous variables: include these variables in the DataFrame you pass to the `forecast` method and provide the future values of these exogenous features over the forecast horizon using the `X_df` parameter.

In [ ]:
#| echo: false
if not IN_COLAB:
    load_dotenv()
    colab_badge('docs/capabilities/forecast/02_exogenous_variables')

In [ ]:
import pandas as pd
from nixtla import NixtlaClient

In [ ]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

> 👍 Use an Azure AI endpoint
> 
> To use an Azure AI endpoint, set the `base_url` argument:
> 
> `nixtla_client = NixtlaClient(base_url="you azure ai endpoint", api_key="your api_key")`

In [ ]:
#| hide
if not IN_COLAB:
    nixtla_client = NixtlaClient()

## 1. Historical exogenous variables

In [ ]:
# Read data
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')

# Forecast
forecast_df = nixtla_client.forecast(
    df=df, 
    h=24,
    id_col='unique_id',
    target_col='y',
    time_col='ds',
    # Add the columns of `df` that will be considered as historical
    hist_exog_list=['Exogenous1', 'Exogenous2', 'day_0', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6']
)


## 2. Future exogenous variables

In [ ]:
# Read data
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')

# Load the future value of exogenous variables over the forecast horizon
future_ex_vars_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-future-ex-vars.csv')

# Forecast
forecast_df = nixtla_client.forecast(
    df=df, 
    X_df=future_ex_vars_df, 
    h=24,
    id_col='unique_id',
    target_col='y',
    time_col='ds'
)


## 3. Historical and future exogenous variables

In [ ]:
# Read data
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')

# Load the future value of exogenous variables over the forecast horizon
future_ex_vars_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-future-ex-vars.csv')

# We will only use 2 exogenous of future_ex_vars_df
future_ex_vars_df = future_ex_vars_df[["unique_id", "ds", "Exogenous1", "Exogenous2"]]
# To pass historical exogenous variables, we need to add the list of columns
# in the `hist_exog_list` as follows.

# Forecast
forecast_df = nixtla_client.forecast(
    df=df, 
    X_df=future_ex_vars_df, 
    h=24,
    id_col='unique_id',
    target_col='y',
    time_col='ds',
    # Add the columns of `df` that will be considered as historical
    hist_exog_list=['day_0', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6']
)

> 📘 Available models in Azure AI
>
> If you use an Azure AI endpoint, set `model="azureai"`
>
> `nixtla_client.detect_anomalies(..., model="azureai")`
>
> For the public API, two models are supported: `timegpt-1` and `timegpt-1-long-horizon`.
> 
> By default, `timegpt-1` is used. See [this tutorial](https://docs.nixtla.io/docs/tutorials-long_horizon_forecasting) for details on using `timegpt-1-long-horizon`.

For more details on using exogenous features with TimeGPT, read our in-depth tutorials on [Exogenous variables](https://docs.nixtla.io/docs/tutorials-exogenous_variables) and on [Categorical variables](https://docs.nixtla.io/docs/tutorials-categorical_variables).